In [9]:
from torchvision.datasets import CelebA
import numpy as np
import os
import torch
import clip
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import copy
import pandas as pd 
import matplotlib.pyplot as plt
from IPython.display import Image, display
%matplotlib inline
import sys
sys.path.insert(0, '../')
import utils as ut
import importlib
from sklearn.metrics import accuracy_score

In [10]:
importlib.reload(ut)

<module 'utils' from '/mnt/efs/fairclip/FinalCode/CelebA/../utils.py'>

In [11]:
device = "cuda:1" #if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)
model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [12]:
projection_GT,projection_inferred, MI_GT, MI_inferred = ut.calculate_projections_celeba(model, preprocess, device)

Files already downloaded and verified


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1628/1628 [15:10<00:00,  1.79it/s]


Error of predicting gender train = 0.01


In [13]:
data = ut.get_CelebA("test", model, preprocess, device)

Files already downloaded and verified


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:48<00:00,  1.84it/s]


In [14]:
classification_tasks = {}
classification_tasks['gender'] = [
    

    ('not wearing glasses','wearing glasses'),
    ('not wearing a necklace', 'wearing a necklace'),
    ('with straight hair', 'with wavy hair'),

]
gt_label = [ 'glasses', 'necklace',  'wavy_hair']


In [15]:
for attr in ['gender']:
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    temp = np.zeros((len(classification_tasks[attr]),2))
    accs = []
    for cc, task in enumerate(classification_tasks[attr]):
        text_inputs = torch.cat([clip.tokenize(f"a photo of a person {word}.") for word in task]).to(device)
        with torch.no_grad():
            text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = (100.0 * data['features'] @ text_features.T).softmax(dim=-1).cpu().numpy().astype(np.float64)
        predictions = np.argmax(similarity,axis=1)
        accs.append(np.around(np.mean(predictions == data['labels'][gt_label[cc]]),2))
        print(task,np.around(np.mean(predictions == data['labels'][gt_label[cc]]),2), np.mean(predictions) )
        for ell in range(2):
#             print(ell)
            idx_ = np.where(data['labels']['gender'] == ell)
            
#             temp[cc, ell] = np.around(np.mean(predictions[idx_] == data['labels'][gt_label[cc]][idx_]),2)
            
            
            idx_tp = np.where(np.logical_and(data['labels'][gt_label[cc]][idx_] == 1, predictions[idx_] == 1))[0]
            
            pos_ = np.where(data['labels'][gt_label[cc]][idx_] == 1)[0].shape[0]
            temp[cc, ell] = idx_tp.shape[0]/ pos_
            
#             temp[cc, ell] = 1 - np.around(np.mean(predictions[data['labels']['gender']== ell] != data['labels'][]),2)
    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = abs(temp['Male'] - temp['Female'])
        temp['Accuracy'] = accs
    temp.to_csv(f"../results_csv/{attr}_celeba_clf_orig.csv")
    print(temp)

--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
('not wearing glasses', 'wearing glasses') 0.98 0.06397154593728083
('not wearing a necklace', 'wearing a necklace') 0.74 0.239655345155796
('with straight hair', 'with wavy hair') 0.59 0.5026049494038674
                                                Female      Male  Disparity  \
(not wearing glasses, wearing glasses)        0.946372  0.790123   0.156249   
(not wearing a necklace, wearing a necklace)  0.447148  0.105691   0.341457   
(with straight hair, with wavy hair)          0.580656  0.852254   0.271598   

                                              Accuracy  
(not wearing glasses, wearing glasses)            0.98  
(not wearing a necklace, wearing a necklace)      0.74  
(with straight hair, with wavy hair)              0.59  


In [8]:
# --- Evaluation of zero-shot classification w.r.t. gender  -------------------------
# Numbers are the mean prediction rate for the first word when classifying into the two words
# ('not wearing glasses', 'wearing glasses') 0.98 0.06397154593728083
# ('not wearing a necklace', 'wearing a necklace') 0.74 0.239655345155796
# ('with straight hair', 'with wavy hair') 0.59 0.5026049494038674
#                                                 Female      Male  Disparity  \
# (not wearing glasses, wearing glasses)        0.946372  0.790123   0.156249   
# (not wearing a necklace, wearing a necklace)  0.447148  0.105691   0.341457   
# (with straight hair, with wavy hair)          0.580656  0.852254   0.271598   

#                                               Accuracy  
# (not wearing glasses, wearing glasses)            0.98  
# (not wearing a necklace, wearing a necklace)      0.74  
# (with straight hair, with wavy hair)              0.59 

SyntaxError: invalid syntax (4139814366.py, line 2)

In [ ]:
('not wearing glasses', 'wearing glasses') 0.98 0.07524296162709147
('not wearing a necklace', 'wearing a necklace') 0.71 0.3014727983168019
('with straight hair', 'with wavy hair') 0.59 0.5050596132652039
                                                Female      Male  Disparity  \
(not wearing glasses, wearing glasses)        0.968454  0.881687   0.086767   
(not wearing a necklace, wearing a necklace)  0.545247  0.186992   0.358255   
(with straight hair, with wavy hair)          0.593014  0.858932   0.265918   

                                              Accuracy  
(not wearing glasses, wearing glasses)            0.98  
(not wearing a necklace, wearing a necklace)      0.71  
(with straight hair, with wavy hair)              0.59

In [16]:
# FPCA GT
from scipy.special import softmax

for attr in ['gender']:
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    temp = np.zeros((len(classification_tasks[attr]),2))
    accs = []
    for cc, task in enumerate(classification_tasks[attr]):
        text_inputs = torch.cat([clip.tokenize(f"a photo of a person {word}.") for word in task]).to(device)
        with torch.no_grad():
            text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        projection_train = projection_GT[attr]
        all_features_val_transf = projection_train.just_transform(data['features'].cpu().numpy().astype(np.float64))
        text_features_pca = projection_train.just_transform(text_features.cpu().numpy().astype(np.float64))
        similarity = softmax(100.0 * np.matmul(all_features_val_transf, np.transpose(text_features_pca)),axis=1)
        
        predictions = np.argmax(similarity,axis=1)
        print(task,np.around(np.mean(predictions == data['labels'][gt_label[cc]]),2), np.mean(predictions), accuracy_score(predictions,data['labels'][gt_label[cc]]))
        accs.append(np.around(np.mean(predictions == data['labels'][gt_label[cc]]),2))
        for ell in range(2):
            idx_ = np.where(data['labels']['gender'] == ell)
            
#             temp[cc, ell] = np.around(np.mean(predictions[idx_] == data['labels'][gt_label[cc]][idx_]),2)
            
            
            idx_tp = np.where(np.logical_and(data['labels'][gt_label[cc]][idx_] == 1, predictions[idx_] == 1))[0]
            
            pos_ = np.where(data['labels'][gt_label[cc]][idx_] == 1)[0].shape[0]
            temp[cc, ell] = idx_tp.shape[0]/ pos_
            
    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = abs(temp['Male'] - temp['Female'])
        temp['Accuracy'] = accs
    elif attr == 'race':
        temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
    temp.to_csv(f"../results_csv/{attr}_celeba_clf_fpca_gt.csv")
    print(temp)

--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
('not wearing glasses', 'wearing glasses') 0.99 0.05901212303376415 0.985622683097886
('not wearing a necklace', 'wearing a necklace') 0.81 0.08726580502955615 0.8128944995491434
('with straight hair', 'with wavy hair') 0.62 0.5875162809337742 0.6198276725778981
                                                Female      Male  Disparity  \
(not wearing glasses, wearing glasses)        0.927445  0.818930   0.108515   
(not wearing a necklace, wearing a necklace)  0.136502  0.170732   0.034230   
(with straight hair, with wavy hair)          0.782336  0.797162   0.014825   

                                              Accuracy  
(not wearing glasses, wearing glasses)            0.99  
(not wearing a necklace, wearing a necklace)      0.81  
(with straight hair, with wavy hair)              0.62  


In [ ]:
np.mean(temp.Disparity)

In [17]:
# FPCA INF
from scipy.special import softmax

for attr in ['gender']:
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    temp = np.zeros((len(classification_tasks[attr]),2))
    accs =[]
    for cc, task in enumerate(classification_tasks[attr]):
        text_inputs = torch.cat([clip.tokenize(f"a photo of a person {word}.") for word in task]).to(device)
        
        with torch.no_grad():
            text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        projection_train = projection_inferred[attr]
        all_features_val_transf = projection_train.just_transform(data['features'].cpu().numpy().astype(np.float64))
        text_features_pca = projection_train.just_transform(text_features.cpu().numpy().astype(np.float64))
        similarity = softmax(100.0 * np.matmul(all_features_val_transf, np.transpose(text_features_pca)),axis=1)
        
        predictions = np.argmax(similarity,axis=1)
        accs.append(np.around(np.mean(predictions == data['labels'][gt_label[cc]]),2))
        print(task,np.around(np.mean(predictions == data['labels'][gt_label[cc]]),2), np.mean(predictions) )
        
        for ell in range(2):
            idx_ = np.where(data['labels']['gender'] == ell)
            
#             temp[cc, ell] = np.around(np.mean(predictions[idx_] == data['labels'][gt_label[cc]][idx_]),2)
            
            
            idx_tp = np.where(np.logical_and(data['labels'][gt_label[cc]][idx_] == 1, predictions[idx_] == 1))[0]
            
            pos_ = np.where(data['labels'][gt_label[cc]][idx_] == 1)[0].shape[0]
            temp[cc, ell] = idx_tp.shape[0]/ pos_
            
            
    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = abs(temp['Male'] - temp['Female'])
        temp['Accuracy'] = accs
    elif attr == 'race':
        temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
    temp.to_csv(f"../results_csv/{attr}_celeba_clf_fpca_inf.csv")
    print(temp)

--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
('not wearing glasses', 'wearing glasses') 0.99 0.05901212303376415
('not wearing a necklace', 'wearing a necklace') 0.81 0.08726580502955615
('with straight hair', 'with wavy hair') 0.62 0.5875663761146178
                                                Female      Male  Disparity  \
(not wearing glasses, wearing glasses)        0.927445  0.818930   0.108515   
(not wearing a necklace, wearing a necklace)  0.136502  0.170732   0.034230   
(with straight hair, with wavy hair)          0.782831  0.795492   0.012662   

                                              Accuracy  
(not wearing glasses, wearing glasses)            0.99  
(not wearing a necklace, wearing a necklace)      0.81  
(with straight hair, with wavy hair)              0.62  


In [ ]:
np.mean(temp.Disparity)

In [18]:
print("======== Running CLF MI G.T on the model ============== ")

for attr in ['gender']:
    num_clip_s = [400, 256]
    mis = MI_GT[attr]
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    for num_clip in num_clip_s:
        print(f"----------- {num_clip}--------------")
        temp = np.zeros((len(classification_tasks[attr]),2))
        accs = []
        for cc, task in enumerate(classification_tasks[attr]):
            text_inputs = torch.cat([clip.tokenize(f"a photo of a person {word}.") for word in task]).to(device)
            with torch.no_grad():
                text_features = model.encode_text(text_inputs)
            text_features_mi =text_features.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            image_features_val = data['features'].cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            similarity = softmax(100.0 * np.matmul(image_features_val, np.transpose(text_features_mi)),axis=1)

            predictions = np.argmax(similarity,axis=1)
            print(task,np.around(np.mean(predictions == data['labels'][gt_label[cc]]),2), np.mean(predictions) )
            accs.append(np.around(np.mean(predictions == data['labels'][gt_label[cc]]),2))
            
            for ell in range(2):
                idx_ = np.where(data['labels']['gender'] == ell)
            
#             temp[cc, ell] = np.around(np.mean(predictions[idx_] == data['labels'][gt_label[cc]][idx_]),2)
            
            
                idx_tp = np.where(np.logical_and(data['labels'][gt_label[cc]][idx_] == 1, predictions[idx_] == 1))[0]

                pos_ = np.where(data['labels'][gt_label[cc]][idx_] == 1)[0].shape[0]
                temp[cc, ell] = idx_tp.shape[0]/ pos_
                
        columns= ['Female', 'Male']
        temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
        if attr == 'gender':	  
            temp['Disparity'] = abs(temp['Male'] - temp['Female'])
            temp['Accuracy'] = accs
        elif attr == 'race':
            temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
        temp.to_csv(f"../results_csv/{attr}_celeba_clf_MI_gt{num_clip}.csv")
        print(temp)
        print(np.mean(temp.Disparity))

======== Running CLF MI G.T on the model ============== 
--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
----------- 400--------------
('not wearing glasses', 'wearing glasses') 0.95 0.013926460274521591
('not wearing a necklace', 'wearing a necklace') 0.86 5.009518084360284e-05
('with straight hair', 'with wavy hair') 0.6 0.6149183448552249
                                                Female      Male  Disparity  \
(not wearing glasses, wearing glasses)        0.186120  0.219136   0.033016   
(not wearing a necklace, wearing a necklace)  0.000000  0.000000   0.000000   
(with straight hair, with wavy hair)          0.795848  0.820534   0.024686   

                                              Accuracy  
(not wearing glasses, wearing glasses)            0.95  
(not wearing a necklace, wearing a necklace)      0.86  
(with straight hair, with wavy hair)    

In [ ]:
np.mean(temp.Disparity)

In [19]:
print("======== Running CLF MI INF on the model ============== ")

for attr in ['gender']:
    num_clip_s = [400, 256]
    mis = MI_inferred[attr]
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    for num_clip in num_clip_s:
        print(f"----------- {num_clip}--------------")
        temp = np.zeros((len(classification_tasks[attr]),2))
        accs = []
        
        for cc, task in enumerate(classification_tasks[attr]):
            text_inputs = torch.cat([clip.tokenize(f"a photo of a person {word}.") for word in task]).to(device)
            with torch.no_grad():
                text_features = model.encode_text(text_inputs)
            text_features_mi =text_features.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            image_features_val = data['features'].cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            similarity = softmax(100.0 * np.matmul(image_features_val, np.transpose(text_features_mi)),axis=1)
            
            
            predictions = np.argmax(similarity,axis=1)
            accs.append(np.around(np.mean(predictions == data['labels'][gt_label[cc]]),2))
            for ell in range(2):
                idx_ = np.where(data['labels']['gender'] == ell)
            
#             temp[cc, ell] = np.around(np.mean(predictions[idx_] == data['labels'][gt_label[cc]][idx_]),2)
            
            
                idx_tp = np.where(np.logical_and(data['labels'][gt_label[cc]][idx_] == 1, predictions[idx_] == 1))[0]

                pos_ = np.where(data['labels'][gt_label[cc]][idx_] == 1)[0].shape[0]
                temp[cc, ell] = idx_tp.shape[0]/ pos_
                
        columns= ['Female', 'Male']
        temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
        if attr == 'gender':	  
            temp['Disparity'] = abs(temp['Male'] - temp['Female'])
            temp['Accuracy'] = accs
            
        elif attr == 'race':
            temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
        temp.to_csv(f"../results_csv/{attr}_celeba_clf_MI_inf{num_clip}.csv")
        print(temp)

======== Running CLF MI INF on the model ============== 
--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
----------- 400--------------
                                                Female      Male  Disparity  \
(not wearing glasses, wearing glasses)        0.186120  0.204733   0.018613   
(not wearing a necklace, wearing a necklace)  0.000000  0.000000   0.000000   
(with straight hair, with wavy hair)          0.790575  0.824708   0.034133   

                                              Accuracy  
(not wearing glasses, wearing glasses)            0.95  
(not wearing a necklace, wearing a necklace)      0.86  
(with straight hair, with wavy hair)              0.60  
----------- 256--------------
                                                Female      Male  Disparity  \
(not wearing glasses, wearing glasses)        0.949527  0.910494   0.039033   
(not

In [ ]:
np.mean(temp.Disparity)